In [2]:
# !wget https://huggingface.co/datasets/huseinzol05/Malay-TTS-Yasmin/resolve/main/tts-malay-yasmin.tar.gz
# !wget https://huggingface.co/datasets/huseinzol05/Malay-TTS-Yasmin/resolve/main/normalized-texts.json
# !tar -xf tts-malay-yasmin.tar.gz
# !rm tts-malay-yasmin.tar.gz

In [4]:
# !wget https://huggingface.co/datasets/huseinzol05/Malay-TTS-Yasmin/resolve/main/tts-malay-yasmin-parliament.tar.gz
# !wget https://huggingface.co/datasets/huseinzol05/Malay-TTS-Yasmin/resolve/main/normalized-parliaments.json
# !tar -xf tts-malay-yasmin-parliament.tar.gz
# !rm tts-malay-yasmin-parliament.tar.gz

In [5]:
import os
import malaya_speech
from malaya_speech import Pipeline
from tqdm import tqdm
import numpy as np
import soundfile as sf
from glob import glob

In [6]:
config = {'sampling_rate': 22050,
 'fft_size': 1024,
 'hop_size': 256,
 'win_length': None,
 'window': 'hann',
 'num_mels': 80,
 'fmin': 0,
 'fmax': None,
 'global_gain_scale': 1.0,
 'trim_silence': True}

In [7]:
directory = 'output-yasmin'
os.system(f'mkdir {directory}')
directories = ['audios']
for d in directories:
    os.system(f'mkdir {directory}/{d}')

In [8]:
import json

with open('normalized-texts.json') as fopen:
    texts = json.load(fopen)
    
with open('normalized-parliaments.json') as fopen:
    parliament = json.load(fopen)

In [9]:
def process(txts, 
            start_silent_trail = int(0.05 * config['sampling_rate']),
            middle_silent_trail = int(0.12 * config['sampling_rate']),
            end_silent_trail = int(0.1 * config['sampling_rate']),
            process_middle_silent = True):
    
    txts = txts[0]
    vad = malaya_speech.vad.webrtc()

    for f in txts:
        directory = f[2]
        index = f[1]
        f = f[0]
        
        audio, _ = malaya_speech.load(f, sr = config['sampling_rate'])
        audio = audio[start_silent_trail:]

        if config['trim_silence']:
            y_= malaya_speech.resample(audio, config['sampling_rate'], 16000)
            y_ = malaya_speech.astype.float_to_int(y_)
            frames = list(malaya_speech.generator.frames(audio, 30, config['sampling_rate']))
            frames_ = list(malaya_speech.generator.frames(y_, 30, 16000, append_ending_trail = False))
            frames_webrtc = [(frames[no], vad(frame)) for no, frame in enumerate(frames_)]
            grouped_deep = malaya_speech.group.group_frames(frames_webrtc)
            grouped_deep = malaya_speech.group.group_frames_threshold(grouped_deep, 0.15)
            r = []
            for no, g in enumerate(grouped_deep):
                if g[1]:
                    g = g[0].array
                else:
                    if no == 0:
                        g = g[0].array[-start_silent_trail:]
                    elif no == (len(grouped_deep) - 1):
                        g = g[0].array[:end_silent_trail]
                    else:
                        if process_middle_silent:
                            g = np.concatenate([g[0].array[:middle_silent_trail], g[0].array[-middle_silent_trail:]])
                        else:
                            g = g[0].array
                        
                r.append(g)
            audio = np.concatenate(r)
        
        
        sf.write(f'{directory}/audios/{index}.wav', audio, config['sampling_rate'])

In [10]:
txts = [(f"female/{t['index']}.wav", t['index'], directory) for t in texts]

In [11]:
i = 1508
process((txts[i: i + 10], 0))

In [12]:
from glob import glob

wavs = glob(f'{directory}/audios/*.wav')
wavs[:10]

['output-yasmin/audios/1509.wav',
 'output-yasmin/audios/1511.wav',
 'output-yasmin/audios/1517.wav',
 'output-yasmin/audios/1514.wav',
 'output-yasmin/audios/1513.wav',
 'output-yasmin/audios/1512.wav',
 'output-yasmin/audios/1516.wav',
 'output-yasmin/audios/1508.wav',
 'output-yasmin/audios/1515.wav',
 'output-yasmin/audios/1510.wav']

In [14]:
import IPython.display as ipd

ipd.Audio(wavs[0])

In [16]:
import mp

for i in tqdm(range(0, len(txts), 1000)):
    index = min(i + 1000, len(txts))
    b = txts[i: index]
    mp.multiprocessing(b, process, cores = 15, returned = False)

100%|██████████| 50/50 [10:52<00:00, 13.05s/it]


In [17]:
len(glob(f'{directory}/audios/*.wav'))

50000

In [18]:
directory = 'output-yasmin-parliament'
os.system(f'mkdir {directory}')
directories = ['audios']
for d in directories:
    os.system(f'mkdir {directory}/{d}')

In [19]:
txts = [(f"female-parliament/{t['index']}.wav", t['index'], directory) for t in parliament]

In [20]:
import mp

for i in tqdm(range(0, len(txts), 1000)):
    index = min(i + 1000, len(txts))
    b = txts[i: index]
    mp.multiprocessing(b, process, cores = 15, returned = False)

100%|██████████| 60/60 [12:18<00:00, 12.31s/it]


In [21]:
directory = 'output-yasmin'
wavs = glob(f'/home/husein/speech-bahasa/{directory}/audios/*.wav')
yasmin = []

for f in wavs:
    left = f
    index = int(os.path.split(f)[1].replace('.wav', ''))
    right = texts[index]['normalized']
    yasmin.append((left, right))
         
yasmin[:2]

[('/home/husein/speech-bahasa/output-yasmin/audios/9176.wav',
  'Beliau juga menyusun kurikulum Sekolah - Sekolah Rendah di Kedah .'),
 ('/home/husein/speech-bahasa/output-yasmin/audios/31535.wav',
  'Ainina turut menyifatkan Mukhriz sebagai pemimpin muda paling gagal dan tidak layak mendepani masa depan generasi milennial Malaysia " .')]

In [22]:
parliament_dict = {i['index']: i for i in parliament}

In [24]:
directory = 'output-yasmin-parliament'
wavs = glob(f'/home/husein/speech-bahasa/{directory}/audios/*.wav')

for f in wavs:
    left = f
    try:
        index = int(os.path.split(f)[1].replace('.wav', ''))
        right = parliament_dict[index]['normalized']
        yasmin.append((left, right))
    except Exception as e:
        print(e)

In [26]:
len(yasmin)

109601

In [28]:
from sklearn.model_selection import train_test_split

yasmin_train, yasmin_test = train_test_split(yasmin, test_size = 2000)

In [29]:
with open('yasmin-vits-test-set.txt', 'w') as fopen:
    json.dump(yasmin_test, fopen)

In [30]:
with open('yasmin-vits-train-set.txt', 'w') as fopen:
    json.dump(yasmin_train, fopen)